In [2]:
#生成字典库
vocab=set([line.rstrip() for line in open('vocab.txt')])

In [3]:
vocab

{'Ashamed',
 'Soothsayer',
 'Sumatra',
 'further',
 'GAF',
 'receded',
 'Hospitals',
 'suits',
 '4.4',
 'Grenoble',
 'Aggies',
 'Kochanek',
 'feat',
 'metre',
 'Husky',
 'tie-up',
 'waterflows',
 'fleshy',
 'equally',
 'individualized',
 'leapfrog',
 'Blyth',
 'Enfield',
 'provincialism',
 'Christian',
 'Excellent',
 'HIKES',
 'primed',
 'eyes',
 '46.8',
 'dipole',
 'charities',
 'Syllabicity',
 'freights',
 'Seashore',
 'Senium',
 'Fat',
 'bodyweight',
 'thesis',
 'ramble',
 'realm',
 'Fox',
 'sets',
 'chips',
 'mining',
 'Harbert',
 'Asahi',
 'taskmaster',
 'overlooking',
 'quantitatively',
 'coop',
 'infamous',
 'neighbourhood',
 'occupies',
 'reckoning',
 'rend',
 'parboiled',
 'habitable',
 'Dyke',
 'roomful',
 'Dining',
 'Lindemann',
 'feathery',
 'ecumenist',
 'Reese',
 'Boismassif',
 'hypnotically',
 'perspectives',
 'Revere',
 'Poldowski',
 'Secrecy',
 'vouchsafes',
 'France',
 'incitement',
 'assessment',
 'hoof',
 'obstinate',
 'Andromache',
 'can',
 'extremity',
 'consummat

In [47]:
def sub_generate_candidates(word,vocab):
    #生成编辑距离为1的单词
    #替换，删除，插入
    #appl;bppl,cppl
    #    :ppl,apl
    #    :appla,applb
    
    letters='abcdefghijklmnopqrstuvwxyz'#使用26个字母处理
    
    splits=[(word[:i],word[i:]) for i in range(len(word)+1)]#对该单词进行拆分
    #appl执行后返回('',appl),('a','ppl'),('ap','pl'),('app','l'),('appl','')
    
    #insert
    insert_list=[]
    for l,r in splits:
        for c in letters:
            insert_list.append(l+c+r)
            
    #delete，删除右边第一个字符,保证右边存在字符
    delete_list=[]
    for l,r in splits:
        if r!='':
            delete_list.append(l+r[1:])
            
    #replace，替换右边第一个字符,保证右边存在字符
    replace_list=[]
    for l,r in splits:
        for c in letters:
            if r!='':
                replace_list.append(l+c+r[1:])
    
    candidates=set(insert_list+delete_list+replace_list)
    
    return candidates

In [49]:
#生成所有的候选集合
def generate_candidates(word,vocab,d):
    #输入一个错误单词word，返回在字典中的所有候选集
    candidates=sub_generate_candidates(word,vocab)
    
    for i in range(1,d):
        for w in candidates:
            candidates=candidates|sub_generate_candidates(w,vocab)
    
    
    #过滤掉不存在于字典中的单词
    return [c for c in candidates if c in vocab]

#generate_candidates('word',vocab,2)

In [4]:
from nltk.corpus import reuters

#读取语料库
categories=reuters.categories()
corpus=reuters.sents(categories=categories)

In [8]:
' '.join(corpus[0])

"ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said ."

In [6]:
#构建语言模型:bigram
term_count={}#每个单词出现的次数
bigram_count={}#每个bigram出现的次数
for doc in corpus:
    doc=['<s>']+doc
    for i in range(0,len(doc)-1):
        #bigram:[i,i+1]
        term=doc[i]
        bigram=doc[i:i+2]
        bigram=' '.join(bigram)
        term_count[term]=term_count.get(term,0)+1
        bigram_count[bigram]=bigram_count.get(bigram,0)+1

In [42]:
term_count

{'<s>': 54716,
 'ASIAN': 12,
 'EXPORTERS': 46,
 'FEAR': 2,
 'DAMAGE': 13,
 'FROM': 208,
 'U': 6388,
 '.': 45900,
 'S': 6382,
 '.-': 167,
 'JAPAN': 295,
 'RIFT': 1,
 'Mounting': 1,
 'trade': 2271,
 'friction': 28,
 'between': 1053,
 'the': 58251,
 'And': 227,
 'Japan': 1594,
 'has': 4679,
 'raised': 334,
 'fears': 65,
 'among': 229,
 'many': 297,
 'of': 35979,
 'Asia': 71,
 "'": 11272,
 's': 9298,
 'exporting': 51,
 'nations': 399,
 'that': 7377,
 'row': 49,
 'could': 1431,
 'inflict': 1,
 'far': 275,
 '-': 13705,
 'reaching': 37,
 'economic': 784,
 'damage': 128,
 ',': 72359,
 'businessmen': 56,
 'and': 25043,
 'officials': 836,
 'said': 25221,
 'They': 562,
 'told': 1424,
 'Reuter': 31,
 'correspondents': 7,
 'in': 26478,
 'Asian': 53,
 'capitals': 7,
 'a': 23492,
 'Move': 1,
 'against': 1428,
 'might': 399,
 'boost': 203,
 'protectionist': 86,
 'sentiment': 61,
 'lead': 261,
 'to': 34035,
 'curbs': 36,
 'on': 8556,
 'American': 817,
 'imports': 776,
 'their': 1345,
 'products': 703,


In [21]:
#用户犯错的概率统计
err_prob={}
for line in open('spell-errors.txt'):
    items=line.split(':')
    correct=items[0].strip()
    mistakes=[item.strip() for item in items[1].strip().split(',')]
    err_prob[correct]={}
    for mis in mistakes:
        err_prob[correct][mis]=1.0/len(mistakes)

In [22]:
err_prob

{'raining': {'rainning': 0.5, 'raning': 0.5},
 'writings': {'writtings': 1.0},
 'disparagingly': {'disparingly': 1.0},
 'yellow': {'yello': 1.0},
 'four': {'forer': 0.2,
  'fours': 0.2,
  'fuore': 0.2,
  'fore*5': 0.2,
  'for*4': 0.2},
 'woods': {'woodes': 1.0},
 'hanging': {'haing': 1.0},
 'aggression': {'agression': 1.0},
 'looking': {'loking': 0.1,
  'begining': 0.1,
  'luing': 0.1,
  'look*2': 0.1,
  'locking': 0.1,
  'lucking': 0.1,
  'louk': 0.1,
  'looing': 0.1,
  'lookin': 0.1,
  'liking': 0.1},
 'eligible': {'eligble': 0.3333333333333333,
  'elegable': 0.3333333333333333,
  'eligable': 0.3333333333333333},
 'electricity': {'electrisity': 0.3333333333333333,
  'electricty*2': 0.3333333333333333,
  'electrizity': 0.3333333333333333},
 'scold': {'schold': 0.5, 'skold': 0.5},
 'adaptable': {'adabtable': 1.0},
 'caned': {'canned': 0.5, 'cained': 0.5},
 'immature': {'imature': 1.0},
 "shouldn't": {'shoudln': 0.5, 'shouldnt': 0.5},
 'swivel': {'swival': 1.0},
 'appropriation': {'apro

In [58]:
import numpy as np

V=len(term_count.keys())

f=open('testdata.txt','r')
for line in f:
    #找到错误的单词
    items=line.rstrip().split('\t')
    line=items[2].split()
    #line=['i','like','apple']
    #单词不存在于词库中定义为出错
    for w in line:
        if w not in vocab:
            #w为出错单词
            #1、生成所有候选集合
            candidates=generate_candidates(w,vocab,1)
            #if len(candidates)<1:candidates=generate_candidates(w,vocab,2)
            if len(candidates)<1:continue
            probs=[]
            #对于每一个候选单词
            #score=p(correct)*p(mistake|correct)
            #防止结果过小取log
            #返回一个score最大的candidates
            for candi in candidates:
                prob=0.0
                #计算erro_prob
                if candi in err_prob and w in err_prob[candi]:
                    prob+=np.log(err_prob[candi][w])
                else:
                    prob+=np.log(0.000001)
                #计算语言模型的概率
                #i brought appla computer
                #appla可以替换成apple
                #LM_p(apple)=p(apple|brought)*p(computer|apple)
                #防止过小取log
                
                idx=line.index(w)#获取错误单词所在的位置
                t=''
                if idx==0:t=candi+' '+line[idx+1]
                elif idx==(len(line)-1): t=line[idx-1]+' '+candi
                else:t=line[idx-1]+' '+candi
                    
                if t in bigram_count.keys():
                    #print(items[2][idx-1])
                    prob+=np.log((bigram_count[t]+1.0)/(term_count[t.split(' ')[0]]+V))
                else:
                    prob+=np.log(1.0/V)
                probs.append(prob)
            max_idx=probs.index(max(probs))
            print(w,candidates[max_idx])

protectionst protectionist
products. products
long-run, long-run
gain. gain
17, 17
retaiation retaliation
cost. cost
busines, business
ltMC.T. ltMC.T
U.S., U.S.
Murtha, Murtha
worried. worried
seriousnyss seriousness
aganst against
us, use
named. named
year, year
sewll sell
dlrs, dlrs
world's worlds
largest. largest
markets, markets
importsi imports
Products, Products
Retaliation, Retaliation
Group. Group
Korea's Koreans
Korea, Korea
Japan. Japan
Koreva Korea
U.S., U.S.
1985. 1985
Malaysia, Malaysia
Japn Japan
Kong, Kong
view. view
advantagne advantage
imports. imports
view, views
Lawrenc Lawrence
Mills, Mills
Industry. Industry
imports, imports
sources. sources
disadxantage disadvantage
trade, trade
said. said
market, market
Friday. Friday
said. said
cenntred centred
beef, beef
country. country
trad trade
continue. continue
Liberala Liberal
economy. economy
inoclude include
year. year
program. program
Representetive Representative
Kuroda, Kuroda
MITI, MITI
dispute. dispute
tonnes, ton

palms, palms
said. said
groers growers
year. year
tonne, tonne
oil, oil
coul could
industry, industry
Reuters. Reuters
March, March
said. said
harvest, harvest
soybean, soybean
tonnas tonnes
crop, crop
said. said
slow, slow
1986, 1986
shares, shares
breief brief
Exchange. Exchange
theire there
shares. shares
sharese shares
today, today
close, closet
dlrs. dlrs
Australia, Australia
ash, ash
30. 30
rose. rose
enkded ended
31, 31
1983, 1983
yen. yen
finst finest
1985/86. 1985/86
impuse impose
products. products
bryefcase briefcase
situation, situation
Shimbun. Shimbun
poer peer
forces. forces
time, time
said. said
here. here
sanctions, sanctions
Yeutter. Yeutter
week's weeks
dela delay
tariffs. tariffs
Bell's Bells
yesterday. yesterday
previouslyl previously
Standard. Standard
Court, Court
Standard, Standard
declilned declined
comment. comment
cheaply. cheaply
namd named
sources. sources
moment. moment
Steei Steel
reasonable, reasonable
China. China
planse plans
1996. 1996
capacigy capaci

billiono billion
today. today
afternoon. afternoon
level, level
average. average
Reader's Readers
said. said
profyt profit
1986. 1986
divdend dividend
divdends dividends
retraction. retraction
sebject subject
approval. approval
commol common
1989. 1989
month's months
shareholders, shareholders
quartr quarter
year. year
investigation, investigation
sevred severed
31, 31
1987. 1987
said. said
withomut without
merit. merits
assetc assets
feedstocks, feedstocks
indeustry industry
year, year
1990, 1990
predicted. predicted
cemical chemical
ltGAF, ltGAF
Inc, Inc
petrocemical petrochemical
plants. plants
ltASH, ltASH
buy. buy
period, period
Oreffice, Oreffice
ltDOW, ltDOW
adding, adding
cpacity capacity
now. now
producds products
ones. ones
customers, customers
plastics, plastics
year. year
capacity, capacity
pices prices
year, year
Dosher, Dosher
Houston. Houston
couldn could
pct, pct
said. said
Oreffice, Oreffice
chemial chemical
Houston, Houston
history. history
1985, 1985
dlrs. dlrs
indus

future, future
reported. reported
Baluchp Baluch
bales, bales
year's years
production. production
Morlin Marlin
tariffs. tariffs
aded added
effect. effect
City, City
N.J., N.J.
Hotel, Hotel
Bowlinq Bowling
well. well
leels levels
facility. facility
year's years
fiscsal fiscal
dlrs. dlrs
Alantic Atlantic
hotel/casino, hotel/casino
dirextly directly
previously. previously
Showtboat Showboat
week. week
earnings, earnings
dlrs, dlrs
repoarted reported
1986. 1986
hodding holding
companya company
Ecuador. Ecuador
inaterest interest
biluion billion
20. 20
Ecuador, Ecuador
whic chic
dlrs, dlrs
interst interest
Mirch March
Ecuadr Ecuador
oil, oil
revenues. revenues
announcement, announcement
Securitye Security
receiverd received
cash. cash
Brzil Brazil
ratel rates
1987. 1987
negotiativons negotiations
resme resume
bilion billion
15. 15
significantr significant
continuingn continuing
foilefd foiled
Basra, Basra
unit, unity
attacked. attacked
warplanes, warplanes
hilicopter helicopter
tanks. tank